In [ ]:
# ! pip install --upgrade pip
# ! pip install --user dask[dataframe]
# ! pip install --user numpy scipy matplotlib seaborn boost_histogram pandas uproot awkward-pandas``

In [ ]:
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

import sys

In [ ]:
plt.rcParams.update({
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "figure.titlesize": 16
})

In [ ]:
sys.path.append("/home/belle2/amubarak/Ds2D0enue_Analysis/07-Python_Functions/")

# Prep-Work

### Import Data

In [ ]:
# # In this notebook we only process the main signal and the generic events,
# # for illustration purposes.
# # You can add other backgrounds after if you wish.
# samples = ["Signal","Other","ccbar_lower_D0_dM"]

# DataFrames = {}  # define empty dictionary to hold dataframes

# # Signal:
# # DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
# # Background
# DataFrames[samples[1]] =  uproot.concatenate("/home/belle2/amubarak/C03-Grid/TopoAna.root:FakeD0",library='pd')
# DataFrames[samples[2]] =  uproot.concatenate("/group/belle2/users2022/amubarak/02-Grid/Completed/Ds2D0e-Generic_Ds_032825_0_All.root:Dstree",library='pd')

In [ ]:
# In this notebook we only process the main signal and the generic events,
# for illustration purposes.
# You can add other backgrounds after if you wish.
samples = ["Signal","All","BB","ccbar","ddbar","ssbar","taupair","uubar","uds"]
DataFrames = {}  # define empty dictionary to hold dataframes

# Signal:
DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
# Background
for s in samples[1:]: # loop over samples
    DataFrames[s] =  uproot.concatenate("/group/belle2/users2022/amubarak/TopoAna/Completed_TopoAna/TopoAna_"+ s +".root:Dstree",library='pd')

### Setup
The code below will be used to apply cuts to the data.

In [ ]:
# Electron ID
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['e_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['e_electronID']>=0.95]
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_em_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_em_electronID']>=0.95]

# Photon Conversion
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=0.1]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=0.1]

# Peaking Background Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_diff_D0pi']>=0.15)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# # Vertex Fitting
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_chiProb']>=0.01]

# Dalitz Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["ccbar"]['Ds_pi0veto_M_Correction']>=0.16)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["Signal"]['Ds_pi0veto_M_Correction']>=0.16)]

# Vertex Fit
#----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames[samples[0]]['Ds_chiProb_Ds_rank']==1]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames[samples[1]]['Ds_chiProb_Ds_rank']==1]

# # D0 Invariant Mass
# #-----------------------
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['D0_dM']>=-0.02) & (DataFrames["Signal"]['D0_dM']<=0.02)]
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['D0_dM']>=-0.02) & (DataFrames["ccbar"]['D0_dM']<=0.02)]
# DataFrames["All"] = DataFrames["All"][(DataFrames["All"]['D0_dM']>=-0.02) & (DataFrames["All"]['D0_dM']<=0.02)]

In [ ]:
DataFrames["Signal"].columns.tolist()

# Reverse Cut
Revering the cuts to get the shapes

Reversing Photon Conversion

In [ ]:
print(abs(DataFrames["All"][(DataFrames["All"]['Ds_gammaveto_M_Correction']<=0.06)]['e_genMotherPDG_2']).value_counts(normalize=True,dropna=False).apply(lambda x: f"{x:.6f}"))

In [ ]:
Stacked = False
Density = False
Bins = 50
var = 'Ds_diff_D0pi'
# var = 'D0_mcPDG'
# = > var = 'Ds_gammaveto_M_Correction'
# var = 'Ds_massDifference_0'
Range = [0.1, 0.55]
dM = 0.06
FD = -1
BS = -1
i = 0
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Other$'
label2= r'$D^{0}$'
label3= r'$D^{*0} \rightarrow D^{0} \; \pi^{0} / \gamma$'
label4= r'$D^{*+} \rightarrow D^{0} \; \pi^{+}$'

labels=[label1,label2,label3,label4]
colors=['C5','C4','C1','C2',]
data=[
      DataFrames[Samples][((abs(DataFrames[Samples]['D0_isSignal']).isna()) | ((abs(DataFrames[Samples]['D0_isSignal'])==0))) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(0.07, Range[1],color='gray',alpha=0.2)
# plt.axvline(0.07, ls='--',color='gray')

# Title
#--------
plt.title(r'$m(e_{sig}^{+} e_{ROE}^{-}) \leq 0.06 GeV/c^{2}$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# plt.xlabel(r'$m(e_{sig}^{+} e_{ROE}^{-})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 25000)
plt.legend(loc='upper right')
plt.show()

In [ ]:
Stacked = True
Density = False
Bins = 50
var = 'Ds_gammaveto_M_Correction'
# i = 'Ds_massDifference_0'
Range = [0., 0.4]
dM = 100
FD = -1
BS = -1
i = 0
Samples = "All"
perBin = ((Range[1] - Range[0])/Bins)*1000
print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

label1= r'$Other$'
label2= r'$D^{0}$'
label3= r'$D^{*0} \rightarrow D^{0} \; \pi^{0} / \gamma$'
label4= r'$D^{*+} \rightarrow D^{0} \; \pi^{+}$'

labels=[label1,label2,label3,label4]
colors=['C5','C4','C1','C2',]
data=[
      DataFrames[Samples][((abs(DataFrames[Samples]['D0_isSignal']).isna()) | ((abs(DataFrames[Samples]['D0_isSignal'])==0))) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_NoDstarplusDstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstar0'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      DataFrames[Samples][(abs(DataFrames[Samples]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames[Samples]['D0_isSignal'])==1) & (DataFrames[Samples]['Ds_extraInfo_BkgBDT']>=BS) & (DataFrames[Samples]['Ds_gammaveto_M_Correction']<=dM)][var],
      ]

# factor = 0.7
# plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['Ds_extraInfo_BkgBDT']>=BS)][var]), ls='--', linewidth=1.5)
plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
plt.axvspan(0.06, Range[1],color='gray',alpha=0.2)
plt.axvline(0.06, ls='--',color='gray')

# Title
#--------
# plt.title(r'$BDT \geq 0.531$', loc = "left")
plt.title(r'$\int\mathcal{L}dt\approx\;1444$ fb$^{-1}$', loc = "right")
# Label
#-------
plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
plt.xlabel(r'$m(e_{sig}^{+} e_{ROE}^{-})\;[GeV/c^{2}]$')
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(0, 30000)
plt.legend()
plt.show()

Reversing Electron ID Cut

In [ ]:
# Dstar_Decay = DataFrames["ccbar_re_eID"][(abs(DataFrames["ccbar_re_eID"]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames["ccbar_re_eID"]['D0_isSignal'])==1)]
# Dstar_Decay = Dstar_Decay[Dstar_Decay["Ds_extraInfo_BkgBDT"]>=0.531]

In [ ]:
# Stacked = False
# Density = False
# Bins = 50
# var = 'e_electronID'
# # i = 'Ds_massDifference_0'
# Range = [0., 1.0]
# eID = 0.5
# perBin = ((Range[1] - Range[0])/Bins)*1000
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# label1= r'$D^{*+} \rightarrow D^{0} X$'

# labels=[label1]
# colors = ['C2']
# data=[
#       # DataFrames["ccbar_re_eID"][(abs(DataFrames["ccbar_re_eID"]['Ds_D0_Dstarplus'])==1) & (abs(DataFrames["ccbar_re_eID"]['D0_isSignal'])==1) & (DataFrames["ccbar_re_eID"]['e_electronID']>=eID)][var],
#       Dstar_Decay[var],
#       ]

# plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# # plt.axvspan(0.5,Range[1],color='gray',alpha=0.2)
# plt.axvline(0.5,ls='--',color='gray')

# # Title
# #--------
# plt.title(r'$\bf Generic \; c \bar{c} \; Events$', loc = "left")
# plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# # Label
# #-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$electronID\;(e^{+})$')
# # plt.yscale("log")
# # plt.xscale("log")
# # plt.ylim(0, 15000)
# plt.legend()
# plt.show()

In [ ]:
# len(DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.16) & (DataFrames["Signal"]['e_electronID']>=0.5)])

In [ ]:
# Stacked = False
# Density = True
# Bins = 40
# # var = 'e_electronID'
# var = 'Ds_diff_D0pi'
# # i = 'Ds_massDifference_0'
# Range = [0.16, 0.55]
# eID = 0.5
# perBin = ((Range[1] - Range[0])/Bins)*1000
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# label1= r'$D^{*+} \rightarrow D^{0} X \; (electronID \geq 0.5)$'
# label2= r'$D^{*+} \rightarrow D^{0} X \; (electronID \leq 0.5)$'

# labels=[label1,label2]
# colors = ["#1f77b4",  # Scientific blue
#           "#55A868"]  # Scientific green (soft sage-like)
# data=[
#       Dstar_Decay[(Dstar_Decay['e_electronID']>=eID)][var],
#       Dstar_Decay[(Dstar_Decay['e_electronID']<=eID)][var],
#       ]

# # factor = 0.7
# # plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['e_electronID']<=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['e_electronID']<=BS)][var]), ls='--', linewidth=1.5)
# plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# # plt.axvspan(0.5,Range[0],color='gray',alpha=0.2)
# # plt.axvline(0.5,ls='--',color='gray')

# # Title
# #--------
# plt.title(r'$\bf Generic \; c \bar{c} \; Events$', loc = "left")
# plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# # Label
# #-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# # plt.yscale("log")
# # plt.xscale("log")
# # plt.ylim(0, 2)
# plt.legend()
# plt.show()

In [ ]:
# Stacked = True
# Density = False
# Bins = 40
# # var = 'e_electronID'
# var = 'Ds_diff_D0pi'
# # i = 'Ds_massDifference_0'
# Range = [0.16, 0.55]
# eID = 0.5
# perBin = ((Range[1] - Range[0])/Bins)*1000
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# label1= r'$D^{*0} \rightarrow D^{0} X \; (mcPDG(e^{+})=211)$'
# label2= r'$D^{*+} \rightarrow D^{0} X \; (mcPDG(e^{+}) \neq 211)$'

# labels=[label1,label2]
# colors = ["#1f77b4",
#           "C3"]  # Scientific green (soft sage-like)
# data=[
#       Dstar_Decay[(Dstar_Decay['e_electronID']>=eID) & (Dstar_Decay['e_mcPDG']==211)][var],
#       Dstar_Decay[(Dstar_Decay['e_electronID']>=eID) & ((Dstar_Decay['e_mcPDG']==11) | (Dstar_Decay['e_mcPDG'].isna()))][var]
#       ]

# # factor = 0.7
# # plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['e_electronID']<=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['e_electronID']<=BS)][var]), ls='--', linewidth=1.5)
# plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# # plt.axvspan(0.5,Range[0],color='gray',alpha=0.2)
# # plt.axvline(0.5,ls='--',color='gray')

# # Title
# #--------
# plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# # Label
# #-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# # plt.yscale("log")
# # plt.xscale("log")
# # plt.ylim(0, 2)
# plt.legend()
# plt.show()

In [ ]:
# print(abs(Dstar_Decay[(Dstar_Decay['e_electronID']<=eID) & (Dstar_Decay['Ds_diff_D0pi']>=0.16)]['e_mcPDG']).value_counts(normalize=True,dropna=False).apply(lambda x: f"{x:.6f}"))

Fake $D^{0}$

In [ ]:
# FakeD0 = DataFrames["ccbar_lower_D0_dM"][(abs(DataFrames["ccbar_lower_D0_dM"]['Ds_D0_Other'])==1) | ((abs(DataFrames["ccbar_lower_D0_dM"]['D0_mcPDG'])==421) & (abs(DataFrames["ccbar_lower_D0_dM"]['D0_isSignal'])==0))]
# # FakeD0 = FakeD0[FakeD0['Ds_extraInfo_FakeD0BDT']>=0.5]
# FakeD0 = FakeD0[FakeD0["Ds_extraInfo_BkgBDT"]>=0.531]

In [ ]:
# Stacked = False
# Density = False
# Bins = 50
# var = 'D0_dM'
# Range = [-0.02, 0.2]
# perBin = ((Range[1] - Range[0])/Bins)*1000
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# # label1= r'$Other$'
# label1= r'$Other \; (BDT \geq 0.531)$'

# labels=[label1]
# colors = ['C5']
# data=[
#       FakeD0[var],
#       ]

# plt.hist(data, color=colors, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# plt.axvspan(0.02,0.04,color='gray',alpha=0.2)
# plt.axvline(0.02,ls='--',color='gray')
# plt.axvline(0.04,ls='--',color='gray')

# # Title
# #--------
# plt.title(r'$\bf Generic \; Events$', loc = "left")
# plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# # Label
# #-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$m(D^{0}) - m_{PDG}(D^{0}) \;[GeV/c^{2}]$')
# # plt.yscale("log")
# # plt.xscale("log")
# # plt.ylim(0, 500)
# plt.legend()
# plt.show()

In [ ]:
# Stacked = False
# Density = True
# Bins = 30
# # var = 'e_electronID'
# var = 'Ds_diff_D0pi'
# # i = 'Ds_massDifference_0'
# Range = [0.16, 0.55]
# BD = -1
# perBin = ((Range[1] - Range[0])/Bins)*1000
# print("Width Per Bin: {width:.2f} MeV".format(width = perBin))

# # Labels
# label1 = r'Other ($D^{0}$ Signal Region, $BDT \geq 0.531$)'
# label2 = r'$D^{0}$ Signal Region: Reverse Fake $D^{0}$ BDT'
# label3 = r'$D^{0}$ Signal Region: Reverse Bkg BDT'
# label4 = r'$D^{0}$ Sideband Region: $BDT \geq 0.531$'
# labels = [label1, 
#           label2, 
#           label3, 
#           label4]
# colors = ["C0", 
#           "C1", 
#           "C2", 
#           "C3"]

# data=[
#       DataFrames[samples[1]][(DataFrames[samples[1]]["Ds_extraInfo_BkgBDT"]>=BD)][var],
#       DataFrames["ccbar_lower_D0_dM"][(DataFrames["ccbar_lower_D0_dM"]["D0_dM"]<=0.02) & (DataFrames["ccbar_lower_D0_dM"]["Ds_extraInfo_BkgBDT"]<=0.531)][var],
#       DataFrames["ccbar_lower_D0_dM"][(DataFrames["ccbar_lower_D0_dM"]["D0_dM"]<=0.02) & (DataFrames["ccbar_lower_D0_dM"]["Ds_extraInfo_FakeD0BDT"]<=0.5)][var],
#       DataFrames["ccbar_lower_D0_dM"][(DataFrames["ccbar_lower_D0_dM"]["D0_dM"]>=0.04) & (DataFrames["ccbar_lower_D0_dM"]["Ds_extraInfo_BkgBDT"]>=BD)][var],
#       ]

# # factor = 0.7
# # plt.hist(DataFrames["Signal"][(DataFrames["Signal"]['e_electronID']<=BS)][var], label="Signal", histtype='step', density=Density, bins=Bins, alpha=1, range=Range, weights=factor*np.ones_like(DataFrames["Signal"][(DataFrames["Signal"]['e_electronID']<=BS)][var]), ls='--', linewidth=1.5)
# plt.hist(data, label=labels, density=Density, stacked=Stacked, bins=Bins, alpha=1, histtype='step', linewidth=1.5, range=Range)
# # plt.axvspan(0.5,Range[0],color='gray',alpha=0.2)
# # plt.axvline(0.5,ls='--',color='gray')

# # Title
# #--------
# plt.title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc = "right")
# # Label
# #-------
# plt.ylabel(r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = perBin))
# plt.xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# # plt.yscale("log")
# # plt.xscale("log")
# # plt.ylim(0, 2)
# plt.legend()
# plt.show()

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# Bins = 30
# var = 'Ds_diff_D0pi'
# Range = [0.16, 0.55]
# BD = 0.531
# bin_width = (Range[1] - Range[0]) / Bins
# perBin = bin_width * 1000
# print("Width Per Bin: {width:.2f} MeV".format(width=perBin))

# # Labels
# label1 = r'Other ($D^{0}$ Signal Region, $BDT \geq 0.531$)'
# label2 = r'$D^{0}$ Signal Region: Reverse Fake $D^{0}$ BDT'
# label3 = r'$D^{0}$ Signal Region: Reverse Bkg BDT'
# label4 = r'$D^{0}$ Sideband Region: $BDT \geq 0.531$'
# labels = [label1, 
#           label2, 
#           label3, 
#           label4]
# colors = ["C0", 
#           "C1", 
#           "C2", 
#           "C3"]

# data=[
#       DataFrames[samples[1]][(DataFrames[samples[1]]["Ds_extraInfo_BkgBDT"]>=BD)][var],
#       DataFrames["ccbar_lower_D0_dM"][(DataFrames["ccbar_lower_D0_dM"]["D0_dM"]<=0.02) & (DataFrames["ccbar_lower_D0_dM"]["Ds_extraInfo_BkgBDT"]<=0.531)][var],
#       DataFrames["ccbar_lower_D0_dM"][(DataFrames["ccbar_lower_D0_dM"]["D0_dM"]<=0.02) & (DataFrames["ccbar_lower_D0_dM"]["Ds_extraInfo_FakeD0BDT"]<=0.5)][var],
#       DataFrames["ccbar_lower_D0_dM"][(DataFrames["ccbar_lower_D0_dM"]["D0_dM"]>=0.04) & (DataFrames["ccbar_lower_D0_dM"]["Ds_extraInfo_BkgBDT"]>=BD)][var],
#       ]

# # Histogram binning and scaling
# bin_edges = np.linspace(Range[0], Range[1], Bins + 1)
# target_total = 10000  # Scale all histograms to this

# fig, ax = plt.subplots(figsize=(8, 6))

# for i, d in enumerate(data):
#     counts, _ = np.histogram(d, bins=bin_edges)
#     scale = target_total / np.sum(counts) if np.sum(counts) > 0 else 0
#     scaled = counts * scale

#     ax.step(0.5 * (bin_edges[1:] + bin_edges[:-1]), scaled,
#             where='mid', label=labels[i], color=colors[i], linewidth=1.5)

# # Plot formatting
# ax.set_title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc="right")
# ax.set_xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# ax.set_ylabel(r'$Entries/(\;{:.2f}\;MeV/c^2)$'.format(perBin))
# ax.legend()
# ax.grid(True)
# plt.tight_layout()
# plt.show()


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# Stacked = False
# Density = False  # Set to False to handle custom scaling
# Bins = 50
# var = 'Ds_diff_D0pi'
# Range = [0.16, 0.55]
# BD = -1
# bin_width = (Range[1] - Range[0]) / Bins
# perBin = bin_width * 1000
# print("Width Per Bin: {width:.2f} MeV".format(width=perBin))

# # Labels
# label1 = r'Other ($D^{0}$ Signal Region, $BDT \geq 0.531$)'
# label2 = r'$D^{0}$ Signal Region: Reverse Fake $D^{0}$ BDT'
# label3 = r'$D^{0}$ Signal Region: Reverse Bkg BDT'
# label4 = r'$D^{0}$ Sideband Region: $BDT \geq 0.531$'
# labels = [label1, label2, label3, label4]
# colors = ["C0", "C1", "C2", "C3"]

# # Data (use your actual DataFrames and variables)
# data = [
#     DataFrames[samples[1]][(DataFrames[samples[1]]["Ds_extraInfo_BkgBDT"] >= BD)][var],
#     DataFrames["ccbar_lower_D0_dM"][
#         (DataFrames["ccbar_lower_D0_dM"]["D0_dM"] <= 0.02) &
#         (DataFrames["ccbar_lower_D0_dM"]["Ds_extraInfo_BkgBDT"] <= 0.531)
#     ][var],
#     DataFrames["ccbar_lower_D0_dM"][
#         (DataFrames["ccbar_lower_D0_dM"]["D0_dM"] <= 0.02) &
#         (DataFrames["ccbar_lower_D0_dM"]["Ds_extraInfo_FakeD0BDT"] <= 0.5)
#     ][var],
#     DataFrames["ccbar_lower_D0_dM"][
#         (DataFrames["ccbar_lower_D0_dM"]["D0_dM"] >= 0.04) &
#         (DataFrames["ccbar_lower_D0_dM"]["Ds_extraInfo_BkgBDT"] >= BD)
#     ][var],
# ]

# # Scaling all histograms to same total count (e.g., 10,000)
# target_total = 100000
# bin_edges = np.linspace(Range[0], Range[1], Bins + 1)
# bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])

# fig, ax = plt.subplots(figsize=(8, 6))

# scaled_histograms = []
# scaled_errors = []

# for i, d in enumerate(data):
#     counts, _ = np.histogram(d, bins=bin_edges)
#     scale = target_total / np.sum(counts) if np.sum(counts) > 0 else 0
#     scaled = counts * scale
#     error = np.sqrt(counts) * scale

#     scaled_histograms.append(scaled)
#     scaled_errors.append(error)

#     ax.errorbar(
#         bin_centers, scaled, yerr=error,
#         fmt='o', label=labels[i], color=colors[i],
#         capsize=2, markersize=4, linewidth=1
#     )

# # Labels
# ax.set_title(r'$\int\mathcal{L}dt\approx\;200$ fb$^{-1}$', loc="right")
# ax.set_ylabel(r'$Entries/(\;{:.2f}\;MeV/c^2)$'.format(perBin))
# ax.set_xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# ax.legend()
# ax.grid(True)
# plt.tight_layout()
# plt.show()


In [ ]:
# fig, ax = plt.subplots(figsize=(10, 4))

# ref = scaled_histograms[0]
# ref_err = scaled_errors[0]

# for i in range(1, len(scaled_histograms)):
#     h = scaled_histograms[i]
#     err = scaled_errors[i]
#     with np.errstate(divide='ignore', invalid='ignore'):
#         ratio = np.divide(h, ref, out=np.zeros_like(h), where=ref!=0)
#         ratio_err = ratio * np.sqrt(
#             (err / h)**2 + (ref_err / ref)**2
#         )
#     ax.errorbar(
#         bin_centers, ratio, yerr=ratio_err,
#         fmt='o', label=f'{labels[i]} / {labels[0]}', color=colors[i], capsize=2
#     )

# ax.axhline(1, color='gray', linestyle='--')
# ax.set_ylabel("Ratio to reference")
# ax.set_xlabel(r'$\Delta m(D_s^{+} - D^{0})\;[GeV/c^{2}]$')
# ax.legend()
# ax.grid(True)
# plt.tight_layout()
# plt.show()
